# 3. Cassandra to Arrow

We use some code from the Cassandra server to read the SSTable, but instead of de/serializing to/from CQL, we use an [Arrow IPC stream](http://arrow.apache.org/), which is stored in a columnar format and better suited for analytics.

Data transformations:

1. SSTable on disk
2. Deserialized into Java Object in C* server
3. Client makes request to server (not to C* DB)
4. Data serialized via Arrow IPC stream
5. Sent across network
6. Arrow IPC stream received by client
7. Transformed into Arrow Table / cuDF

**Pros:**
- doesn't make request to the main Cassandra DB, which lessens the load and allows for other operations to run
- less de/serialization involved using the Arrow IPC stream

**Cons:**
- don't want to have to start Cassandra or use the JVM
- complex architecture

In [1]:
import pyarrow as pa
import pandas as pd
import socket

HOST = '127.0.0.1'
PORT = 9143

In [2]:
# read data from socket
def fetch_data():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        s.sendall(b'hello world\n')
        data = b''
        while True:
            newdata = s.recv(1024)
            if not newdata:
                break
            data += newdata
    return data

In [4]:
buffer = fetch_data()
reader = pa.ipc.open_stream(buffer)
arrow_table = reader.read_all()
arrow_table.to_pandas() # for visualization

,liveness_info_tstamp,partition key,value
0,2021-07-15 16:51:35.440203,6,439790106
1,2021-07-15 16:51:35.442510,7,564330072
2,2021-07-15 16:51:35.446255,9,97405552
3,2021-07-15 16:51:35.434989,4,351686621
4,2021-07-15 16:51:35.416599,3,352527683
5,2021-07-15 16:51:35.437366,5,114304900
6,2021-07-15 16:51:35.403305,0,382062539
7,2021-07-15 16:51:35.444276,8,296173906
8,2021-07-15 16:51:35.414324,2,949364593
9,2021-07-15 16:51:35.409490,1,774912474


In [5]:
!ls ../cpp/build

CMakeCache.txt          deletion_time.h         sstable_statistics.cpp
CMakeFiles              results.json            sstable_statistics.h
CPackConfig.cmake       rules.ninja             sstable_summary.cpp
CPackSourceConfig.cmake sstable_data.cpp        sstable_summary.h
build.ninja             sstable_data.h          sstable_to_arrow
cmake_install.cmake     sstable_index.cpp       table.parquet
deletion_time.cpp       sstable_index.h


In [6]:
parquet_table = pd.read_parquet("../cpp/build/table.parquet")

In [7]:
parquet_table

,_timestamp,partition key,clustering key,data,sensor_value,station_id
0,1924-04-12,{'org.apache.cassandra.db.marshal.UUIDType': b...,1970-01-01,"vulputate. Vestibulum at imperdiet metus, et ...",97.651955,b'(\xdfc\xb7\xccWC\xcb\x97R\xfa\xe6\x9d\x16S\xda'
